In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [1]:
def read_meta_data(path):
    df_meta = pd.read_csv(path, nrows=7, index_col=0, header =None).transpose()
    meta_dict = {}
    gridBoundaries = [float(num) for num in df_meta['Data Bounding Box:'].values[0].split(',')]
    df = pd.DataFrame()
    meta_dict['x0'] = gridBoundaries[0]
    meta_dict['y0'] = gridBoundaries[1]
    meta_dict['x1'] = gridBoundaries[2]
    meta_dict['y1'] = gridBoundaries[3]
    meta_dict['Start_date'] = df_meta['User Start Date:'].values
    meta_dict['End_date'] = df_meta['User End Date:'].values
    meta_dict['Title'] = df_meta['Title:'].values
    meta_dict['Url'] = df_meta['URL to Reproduce Results:'].values
    meta_dict['fvalue'] = df_meta['Fill Value (mean_OMNO2d_003_ColumnAmountNO2TropCloudScreened):'].values
    
    
    return meta_dict

## NO2 Data

In [49]:


def gen_html(path2020, path2019, title, savefile):
    meta_dict=read_meta_data
    df2020 = pd.read_csv(path2020, skiprows=9, index_col=0, header=None, names=['NO2(1/cm2)'],na_values=meta_dict['fvalue'])
    df2019 = pd.read_csv(path2019, skiprows=9, index_col=0, header=None, names=['NO2(1/cm2)'],na_values=meta_dict['fvalue'])
    #Remove negative values
    df2020 = df2020[df2020['NO2(1/cm2)'] > 0]
    df2019 = df2019[df2019['NO2(1/cm2)'] > 0]

    rolling_avg = df2020.rolling(7).mean()

    fig = go.Figure()
    fig.add_trace(go.Scatter(y = df2020['NO2(1/cm2)'],
                            x= df2020.index, name='2020' )) 

    fig.add_trace(go.Scatter(y = df2019['NO2(1/cm2)'],
                            x= df2020.index, name ='2019', line =dict(dash='dash'))) 

    fig.update_layout(
        title=title,
        xaxis_title = 'Date',
        yaxis_title = 'NO2 (1/cm^2)',
        hovermode="x")

    fig.write_html(savefile, include_plotlyjs= 'directory')

In [52]:
pathRome2020 = '../data/omi/OMNO2d_003_20200131-20200524_12E_40N_15E_42N.csv'
pathRome2019 = '../data/omi/OMNO2d_003_20190131-20190524_12E_40N_15E_42N.csv'

pathMilan2020 = '../data/omi/OMNO2d_003_20200131-20200524_8E_44N_12E_46N.csv'
pathMilan2019 = '../data/omi/OMNO2d_003_20190131-20190524_8E_44N_12E_46N.csv'

pathMadrid2020 = '../data/omi/OMNO2d_003_20200131-20200524_5W_39N_2W_41N.csv'

pathMadrid2019 = '../data/omi/OMNO2d_003_20190131-20190524_5W_39N_2W_41N.csv'


gen_html(path2020=pathRome2020, path2019=pathRome2019, title='OMI Tropospheric NO2 Rome', 
         savefile='../html/NO2_Rome.html' )

gen_html(pathMadrid2020, pathMadrid2019, title='OMI Tropospheric NO2 Madrid',
        savefile='../html/NO2_madrid.html')

gen_html(pathMilan2020, pathMilan2019, title='OMI Tropospheric NO2 Milian', 
        savefile='../html/NO2_milan.html')

## Flight data

In [87]:
df_flights = pd.read_csv('../data/flightradar/number-of-commercial-fli.csv'
                        , index_col = 0)
df_flights.index = pd.to_datetime(df_flights.index) 

In [103]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_flights.index, y=df_flights['Number of flights'],
             name = 'Daily flights'))

fig.add_trace(go.Scatter(x=df_flights.index, y=df_flights['7-day moving average'],
                        name='7-day moving avarage', line=dict(color ='slateblue'
                                                              ), opacity=0.7))
fig.update_layout(title='Number of commerical flights',
                 xaxis_title='date')
fig.write_html('../html/flight_data.html', include_plotlyjs= 'directory')